
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## Before running this notebook, you must have already the numerically simulated waves associated to the representative cases of synthetic simulated TCs (obtained with MaxDiss algorithm in notebook 06)

inputs required: 
  * Synthetic simulation of historical TCs parameters (copulas obtained in *notebook 06*)
  * MaxDiss selection of synthetic simulated TCs (parameters obtained in *notebook 06*)
  * simulated waves for the above selected TCs (**outside TeslaKit**)
  
in this notebook:
  * RBF's interpolation of wave conditions based on TCs parameters (from swan simulated TCs waves)

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit 
from teslakit.database import Database
from teslakit.rbf import RBF_Reconstruction, RBF_Validation



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')
           

In [3]:
# --------------------------------------
# load data and set parameters

# r2 TCs Copula Simulated (dataset) 
TCs_r2_sim_params = db.Load_TCs_r2_sim_params()

# r2 TCs MDA selection and solved simulations (not solved inside teslakit)
TCs_r2_MDA_params = db.Load_TCs_r2_mda_params()
TCs_sims = db.Load_TCs_r2_mda_Simulations()



## Simulated TCs -  Radial Basis Function

In [4]:
# --------------------------------------
# prepare dataset and subset

#  RBFs training subset (TCs numerically solved)
subset = np.column_stack(
    (TCs_r2_MDA_params['pressure_min'], TCs_r2_MDA_params['velocity_mean'], 
     TCs_r2_MDA_params['gamma'], TCs_r2_MDA_params['delta'])
)

#  RBFs dataset to interpolate 
dataset = np.column_stack(
    (TCs_r2_sim_params['pressure_min'], TCs_r2_sim_params['velocity_mean'], 
     TCs_r2_sim_params['gamma'], TCs_r2_sim_params['delta'])
)


In [5]:
# --------------------------------------
# Extract waves data from TCs simulations (this is the RBFs training target)

print(TCs_sims)
print()

# Normalize data
d_maxis = {}
d_minis = {}
tcp = TCs_sims.copy()
for k in ['hs', 'tp', 'ss', 'twl']:
    v = tcp[k].values[:]
    mx = np.max(v)
    mn = np.min(v)
    tcp[k] =(('storm',), (v-mn)/(mx-mn))
    
    # store maxs and mins for denormalization
    d_maxis[k] = mx
    d_minis[k] = mn

tcp['dir'] = tcp['dir'] * np.pi/180
    
print(tcp)
print()


# Build RBF target numpy array
target = np.column_stack(
    (tcp['hs'], tcp['tp'], tcp['ss'], tcp['twl'], tcp['dir'], tcp['mu'])
)


<xarray.Dataset>
Dimensions:  (storm: 1000)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    hs       (storm) float64 0.13 0.026 4.398 1.352 ... 1.782 8.078 5.11 6.136
    tp       (storm) float64 3.61 1.506 14.71 9.259 ... 6.494 13.16 11.9 13.33
    dir      (storm) float64 42.4 45.0 124.2 267.3 13.2 ... 35.2 23.5 90.0 122.3
    ss       (storm) float64 0.005154 0.007572 0.09592 ... 0.1987 0.07206 0.1122
    twl      (storm) float64 0.07414 0.02044 1.73 0.5898 ... 2.181 1.498 1.863
    mu       (storm) float64 0.7074 0.8419 0.805 0.9491 ... 0.623 0.5043 0.6556

<xarray.Dataset>
Dimensions:  (storm: 1000)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    hs       (storm) float64 0.009134 0.001473 0.3235 ... 0.5946 0.376 0.4515
    tp       (storm) float64 0.1312 0.0 0.823 0.4834 ... 0.7265 0.6484 0.7375
    dir      (storm) float64 0.74 0.7854 2.168 4.665 ...

In [6]:
# --------------------------------------
# RBF Interpolation

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]        # scalar (pmean,  vmean)
ix_directional_subset = [2,3]   # directional (delta, gamma)

# target - scalar / directional indexes
ix_scalar_target = [0,1,2,3,5]  # scalar (Hs, Tp, SS, TWL, MU)
ix_directional_target = [4]     # directional (Dir)

output = RBF_Reconstruction(
    subset, ix_scalar_subset, ix_directional_subset,
    target, ix_scalar_target, ix_directional_target,
    dataset)


ix_scalar: 0,  optimization: 132.34 | interpolation: 14.23
ix_scalar: 1,  optimization: 116.04 | interpolation: 14.09
ix_scalar: 2,  optimization: 170.79 | interpolation: 14.19
ix_scalar: 3,  optimization: 120.90 | interpolation: 13.86
ix_scalar: 5,  optimization: 146.85 | interpolation: 14.89
ix_directional: 4,  optimization: 349.69 | interpolation: 28.95


In [7]:
# --------------------------------------
# Reconstructed TCs

# denormalize RBF output
TCs_RBF_out = xr.Dataset(
    {
        'hs':(('storm',), output[:,0] * (d_maxis['hs']-d_minis['hs']) + d_minis['hs'] ),
        'tp':(('storm',), output[:,1] * (d_maxis['tp']-d_minis['tp']) + d_minis['tp'] ),
        'ss':(('storm',), output[:,2] * (d_maxis['ss']-d_minis['ss']) + d_minis['ss'] ),
        'twl':(('storm',), output[:,3] * (d_maxis['twl']-d_minis['twl']) + d_minis['twl'] ),
        'dir':(('storm',), output[:,4] * 180 / np.pi),
        'mu':(('storm',), output[:,5]),
    },
    coords = {'storm': np.arange(output.shape[0])}
)
print(TCs_RBF_out)

# store data as xarray.Dataset
db.Save_TCs_sim_r2_rbf_output(TCs_RBF_out)


<xarray.Dataset>
Dimensions:  (storm: 100000)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 ... 99994 99995 99996 99997 99998 99999
Data variables:
    hs       (storm) float64 2.225 3.766 2.855 2.383 ... 1.223 6.09 0.4851 5.758
    tp       (storm) float64 6.284 12.51 10.25 7.49 ... 9.564 12.28 5.676 11.93
    ss       (storm) float64 0.04361 0.07181 0.07295 ... 0.1217 0.01199 0.08449
    twl      (storm) float64 0.5226 1.344 0.94 0.6393 ... 1.757 0.2945 1.605
    dir      (storm) float64 134.6 101.2 69.15 63.19 ... 76.73 115.1 86.7 105.3
    mu       (storm) float64 0.8953 0.8146 0.5073 ... 0.6522 0.6225 0.7201


In [8]:
# --------------------------------------
# RBF Validation

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]        # scalar (pmean,  vmean)
ix_directional_subset = [2,3]   # directional (delta, gamma)

# target - scalar / directional indexes
ix_scalar_target = [0,1,2,3,5]  # scalar (Hs, Tp, SS, TWL, MU)
ix_directional_target = [4]     # directional (Dir)

output = RBF_Validation(
    subset, ix_scalar_subset, ix_directional_subset,
    target, ix_scalar_target, ix_directional_target)


RBFs Kfold Validation: 1/3
ix_scalar: 0,  optimization: 49.84 | interpolation: 0.04
ix_scalar: 1,  optimization: 49.22 | interpolation: 0.04
ix_scalar: 2,  optimization: 93.58 | interpolation: 0.06
ix_scalar: 3,  optimization: 64.21 | interpolation: 0.05
ix_scalar: 5,  optimization: 79.62 | interpolation: 0.06
ix_directional: 4,  optimization: 198.56 | interpolation: 0.15
mean squared error : 127.0075925744685

RBFs Kfold Validation: 2/3
ix_scalar: 0,  optimization: 58.15 | interpolation: 0.07
ix_scalar: 1,  optimization: 59.26 | interpolation: 0.07
ix_scalar: 2,  optimization: 81.67 | interpolation: 0.06
ix_scalar: 3,  optimization: 65.24 | interpolation: 0.08
ix_scalar: 5,  optimization: 88.73 | interpolation: 0.06
ix_directional: 4,  optimization: 188.45 | interpolation: 0.18
mean squared error : 62.716481464479536

RBFs Kfold Validation: 3/3
ix_scalar: 0,  optimization: 76.28 | interpolation: 0.06
ix_scalar: 1,  optimization: 88.77 | interpolation: 0.06
ix_scalar: 2,  optimization: